In [1]:
%cd ../src

C:\Users\nozoe-tatsuya\dev\ai-ocr-ensemble\src


In [2]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

from pathlib import Path

import dataprocess as dp
import models
from args import Args

In [3]:
# デフォルト値以外のパラメータ設定
# デフォルト値はArgsの定義を参照

arg_params = {
    'epochs': 200,
    'batch_size': 512,
    'nb_classes': 17,
    'eval_interval': 5,
    # ResNet parameters
    'resnet_layers': 18,
    'use_se_module': False,
    # optimizer
    'optimizer': 'adaberief',
    'lr': 1e-3,
    'weight_decay': 1e-4,
    # scheduler
    'scheduler': 'cosine',
    'warmup': True,
    'warmup_epoch': 5,
    
    'device': 'cuda',
    'log_dir': '../logs',  # logのルートディレクトリ
    'project': 'model-ensemble',  # logファイルのディレクトリ
    'model': 'resnet',
    'version': 'exclude-maxpool',
    'json_path': '../data/dataset_info_1.json'
}

In [4]:
args = Args(**arg_params)
device = torch.device(args.device)
args, device

(Args(batch_size=512, nb_classes=17, epochs=200, ckpt_interval=20, eval_interval=5, device='cuda', log_dir='../logs', model_dir='../models', project='model-ensemble', model='resnet', version='exclude-maxpool', resnet_layers=18, use_se_module=False, activation='ReLU', patch_size=16, num_blocks=30, d_model=128, mlp_ratio=6, json_path='../data/dataset_info_1.json', lr=0.001, weight_decay=0.0001, optimizer='adaberief', final_lr=0.1, scheduler='cosine', warmup=True, warmup_epoch=5),
 device(type='cuda'))

In [5]:
# データセット設定ファイル
json_path = Path(args.json_path)
assert json_path.is_file()

In [6]:
train_loader = dp.build_dataloader('train', args)
val_loader = dp.build_dataloader('val', args)

In [7]:
system = models.build_system(args)

In [8]:
# for TensorBoard
logger = models.get_tensorboard_logger(args)

In [9]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_err_epoch',
    dirpath='../models',
    filename=args.model + '-' + args.version +  '-{epoch:03d}-{valid_err_epoch:.2f}',
    save_top_k=1,
    mode='min',
)

C:\Users\nozoe-tatsuya\Anaconda3\envs\pytorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:52: UserWarning: Checkpoint directory ../models exists and is not empty.
  warnings.warn(*args, **kwargs)


In [10]:
trainer = pl.Trainer(gpus=1, max_epochs=args.epochs, min_epochs=1,
                     logger=logger,
                     check_val_every_n_epoch=args.eval_interval,
                     callbacks=[checkpoint_callback])

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [11]:
trainer.fit(system, train_loader, val_dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief



  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | CrossEntropyLoss | 0     
1 | train_metrics | Accuracy         | 0     
2 | valid_metrics | Accuracy         | 0     
3 | model         | ResNet           | 2.8 M 
---------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.222    Total estimated model params size (MB)


C:\Users\nozoe-tatsuya\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:508: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


1

### View training logs in TensorBoard

```bash
$ tensorboard --logdir ./logs/MYPROJECT/
```

and access to http://localhost:6006/

プロセスを占有するのでJupyterからの起動は非推奨

## Inference

In [11]:
# system = models.load_model_from_checkpoint('../models/model-ensemble-epoch=174-valid_acc_epoch=0.99.ckpt',
#                                              args)

In [12]:
results = models.infer(system, val_loader, device=device)

In [13]:
result_df, conf_matrix_df = models.build_inference(results, args, datatype='val', return_conf_matrix=True)

C:\Users\nozoe-tatsuya\dev\ai-ocr-ensemble\src\functions\conf_matrix.py:38: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  conf_matrix_df.loc[:-1, 'recall'] = recall


In [14]:
conf_matrix_df

推論ラベル                                                      \
                      0        1           2           3           4      5   
正解ラベル     0  393.000000    0.000    2.000000    0.000000    1.000000    0.0   
          1    1.000000  398.000    0.000000    0.000000    0.000000    0.0   
          2    0.000000    0.000  399.000000    0.000000    0.000000    0.0   
          3    1.000000    0.000    0.000000  396.000000    0.000000    0.0   
          4    0.000000    2.000    0.000000    0.000000  391.000000    0.0   
          5    0.000000    0.000    0.000000    0.000000    0.000000  398.0   
          6    3.000000    0.000    0.000000    0.000000    0.000000    0.0   
          7    0.000000    0.000    1.000000    0.000000    0.000000    0.0   
          8    0.000000    0.000    2.000000    1.000000    0.000000    0.0   
          9    0.000000    0.000    0.000000    0.000000    1.000000    0.0   
          *    0.000000    0.000    0.000000    0.000000    0.000000    0.0   
          ×    0.000000    0.000    0.000000    0.000000    0.000000    0.0   
          -    0.000000    0.000    0.000000    0.000000    0.000000    0.0   
          ・    0.000000    0.000    0.000000    0.000000    0.000000    0.0   
          /    1.000000    0.000    1.000000    0.000000    1.000000    0.0   
          字    0.000000    0.000    0.000000    1.000000    0.000000    0.0   
          ―    0.000000    0.000    0.000000    0.000000    0.000000    0.0   
precision      0.984962    0.995    0.985185    0.994975    0.992386    1.0   

                                                                    \
                      6           7           8      9           *   
正解ラベル     0    0.000000    0.000000    0.000000    0.0    1.000000   
          1    0.000000    0.000000    0.000000    0.0    1.000000   
          2    0.000000    0.000000    0.000000    0.0    0.000000   
          3    0.000000    3.000000    0.000000    0.0    0.000000   
          4    0.000000    0.000000    1.000000    0.0    0.000000   
          5    1.000000    0.000000    0.000000    0.0    0.000000   
          6  398.000000    0.000000    0.000000    0.0    0.000000   
          7    0.000000  399.000000    0.000000    0.0    0.000000   
          8    0.000000    0.000000  395.000000    0.0    0.000000   
          9    0.000000    1.000000    0.000000  397.0    0.000000   
          *    0.000000    0.000000    0.000000    0.0  396.000000   
          ×    0.000000    0.000000    0.000000    0.0    2.000000   
          -    0.000000    0.000000    0.000000    0.0    0.000000   
          ・    0.000000    0.000000    0.000000    0.0    0.000000   
          /    0.000000    0.000000    0.000000    0.0    0.000000   
          字    0.000000    0.000000    0.000000    0.0    3.000000   
          ―    0.000000    0.000000    0.000000    0.0    2.000000   
precision      0.997494    0.990074    0.997475    1.0    0.977778   

                                                                               \
                      ×           -      ・           /           字          ―   
正解ラベル     0    0.000000    1.000000    0.0    0.000000    2.000000    0.00000   
          1    0.000000    0.000000    0.0    0.000000    0.000000    0.00000   
          2    0.000000    0.000000    0.0    1.000000    0.000000    0.00000   
          3    0.000000    0.000000    0.0    0.000000    0.000000    0.00000   
          4    0.000000    0.000000    0.0    5.000000    0.000000    0.00000   
          5    0.000000    0.000000    0.0    1.000000    0.000000    0.00000   
          6    0.000000    0.000000    0.0    0.000000    0.000000    0.00000   
          7    0.000000    0.000000    0.0    0.000000    0.000000    0.00000   
          8    0.000000    0.000000    0.0    2.000000    0.000000    0.00000   
          9    0.000000    0.000000    0.0    0.000000    1.000000    0.00000   
          *    1.000000    0.000000    0.0    0.000000    1.0000

In [14]:
conf_matrix_df.to_excel('../results/conf_matrix_resnet_210525.xlsx')

In [15]:
result_df['correct'].mean()

0.9917676994461907

In [16]:
torch.save(system.model.state_dict(), '../models/resnet_epoch200_210616.pth', pickle_protocol=4)